<a href="https://colab.research.google.com/github/Jarmos-san/Skin-Cancer-MNIST/blob/master/Skin_Cancer_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

In [0]:
# Upload the downloaded kaggle.json file
from google.colab import files
import time

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

In [4]:
# Copy the kaggle.json file to the suggested dir
!mv /content/kaggle.json /root/.kaggle/kaggle.json

# Secure access to the kaggle.json file
!chmod 600 /root/.kaggle/kaggle.json

# Remove unecessary dir & files
!rm -rf /content/sample_data

# Check if the Kaggle CLI works fine or not
!kaggle --version

Kaggle API 1.5.4


In [10]:
# Download the zipped file from Kaggle
!kaggle datasets download kmader/skin-cancer-mnist-ham10000

# Unzip downloaded folder to /content dir
!unzip /content/skin-cancer-mnist-ham10000.zip

# remove downloaded zipped folder
!rm /content/skin-cancer-mnist-ham10000.zip

100% 5.19G/5.20G [01:13<00:00, 77.9MB/s]
100% 5.20G/5.20G [01:13<00:00, 76.0MB/s]
